In [1]:
# Install Hugging Face datasets if not already installed
!pip install datasets -q

from datasets import load_dataset
import pandas as pd

# Load dataset
dataset = load_dataset("Dingdong-Inc/FreshRetailNet-50K", split="train")
df = dataset.to_pandas()

# Show the first few rows
df.head()


,city_id,store_id,management_group_id,first_category_id,second_category_id,third_category_id,product_id,dt,sale_amount,hours_sale,stock_hour6_22_cnt,hours_stock_status,discount,holiday_flag,activity_flag,precpt,avg_temperature,avg_humidity,avg_wind_level
0,0,0,0,5,6,65,38,2024-03-28,0.1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, ...",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.0,0,0,1.6999,15.48,73.54,1.97
1,0,0,0,5,6,65,38,2024-03-29,0.1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, ...",1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.0,0,0,3.0190,15.08,76.56,1.71
2,0,0,0,5,6,65,38,2024-03-30,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.0,1,0,2.0942,15.91,76.47,1.73
3,0,0,0,5,6,65,38,2024-03-31,0.1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, ...",11,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, ...",1.0,1,0,1.5618,16.13,77.40,1.76
4,0,0,0,5,6,65,38,2024-04-01,0.2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, ...",8,"[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...",1.0,0,0,3.5386,15.37,78.26,1.25


In [5]:
# Convert 'dt' to datetime
df['dt'] = pd.to_datetime(df['dt'])

# Create new features
df['hour'] = df['dt'].dt.hour
df['day'] = df['dt'].dt.day
df['day_of_week'] = df['dt'].dt.dayofweek
df['week'] = df['dt'].dt.isocalendar().week
df['month'] = df['dt'].dt.month

# Sort values
df = df.sort_values(by=["store_id", "product_id", "dt"])

# Fill missing demand values (if any)
df['sale_amount'] = df['sale_amount'].fillna(0)

# Optional: Convert categorical flags
df['holiday_flag'] = df['holiday_flag'].astype(int)
df['activity_flag'] = df['activity_flag'].astype(int)


In [7]:
# Create lag features
for lag in [1, 2, 3, 6, 12, 24]:
    df[f'demand_lag_{lag}'] = df.groupby(['store_id', 'product_id'])['sale_amount'].shift(lag)

# Drop NA rows after shift
df = df.dropna()


In [11]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

features = ['hour', 'day', 'day_of_week', 'week', 'month',
            'avg_temperature', 'avg_humidity', 'avg_wind_level',
            'holiday_flag', 'activity_flag'] + [f'demand_lag_{l}' for l in [1, 2, 3, 6, 12, 24]]

# Target variable
target = 'sale_amount'

# Train-test split
X = df[features]
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)

# Train model
model = XGBRegressor()
model.fit(X_train, y_train)

# Predict
predictions = model.predict(X_test)

# Evaluate
mae = mean_absolute_error(y_test, predictions)
print(f"Mean Absolute Error: {mae}")


Mean Absolute Error: 0.3531275715606706


In [15]:
# Apply the model on latest rows to forecast
forecast_input = df[features].tail(100)  # last 100 hourly entries
forecast_result = model.predict(forecast_input)

forecast_df = df[['store_id', 'product_id', 'dt']].tail(100).copy()
forecast_df['forecasted_demand'] = forecast_result
forecast_df.head()


,store_id,product_id,dt,forecasted_demand
4498166,897,775,2024-05-23,1.072930
4498167,897,775,2024-05-24,0.844729
4498168,897,775,2024-05-25,1.004837
4498169,897,775,2024-05-26,1.195314
4498170,897,775,2024-05-27,0.803679


In [17]:
# Add current stock levels (mocked here for demo)
import numpy as np
forecast_df['inventory'] = np.random.randint(0, 100, size=len(forecast_df))

# Trigger condition
forecast_df['needs_redistribution'] = forecast_df['forecasted_demand'] > forecast_df['inventory']

# Identify shortages
shortages = forecast_df[forecast_df['needs_redistribution']]
shortages[['store_id', 'product_id', 'inventory', 'forecasted_demand']]


,store_id,product_id,inventory,forecasted_demand
4495844,897,834,0,6.086174


In [23]:
# Add inventory column (mock if missing)
forecast_df['inventory_level'] = 50  # placeholder, use real if available

# Define alert threshold
forecast_df['alert'] = forecast_df['forecasted_demand'] > forecast_df['inventory_level']

# Filter items that may need stock transfer
alerts = forecast_df[forecast_df['alert']]
alerts[['store_id', 'product_id', 'dt', 'forecasted_demand', 'inventory_level']]


,store_id,product_id,dt,forecasted_demand,inventory_level


In [25]:
import pandas as pd
import numpy as np

# Simulate demand forecasts and inventory
np.random.seed(42)
n = 200

forecast_df = pd.DataFrame({
    'store_id': np.random.choice(range(100, 110), size=n),
    'sku_id': np.random.choice(['PROD001', 'PROD002', 'PROD003'], size=n),
    'dt': pd.date_range(start="2025-07-13", periods=n, freq='H'),
    'forecasted_demand': np.random.randint(20, 120, size=n),
    'inventory_level': np.random.randint(10, 100, size=n)
})

# Identify stockout risk
forecast_df['alert'] = forecast_df['forecasted_demand'] > forecast_df['inventory_level']


C:\Users\anuja\AppData\Local\Temp\ipykernel_21412\3068103867.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  'dt': pd.date_range(start="2025-07-13", periods=n, freq='H'),


In [27]:
redistribution = []

for _, alert_row in forecast_df[forecast_df['alert']].iterrows():
    sku = alert_row['sku_id']
    needed_qty = alert_row['forecasted_demand'] - alert_row['inventory_level']
    target_store = alert_row['store_id']
    
    # Find stores with surplus stock of same SKU
    surplus_df = forecast_df[
        (forecast_df['sku_id'] == sku) &
        (forecast_df['inventory_level'] > forecast_df['forecasted_demand'] + 10) &
        (forecast_df['store_id'] != target_store)
    ].sort_values(by='inventory_level', ascending=False)
    
    for _, surplus_row in surplus_df.iterrows():
        move_qty = min(surplus_row['inventory_level'] - surplus_row['forecasted_demand'], needed_qty)
        if move_qty > 0:
            redistribution.append({
                'from_store': surplus_row['store_id'],
                'to_store': target_store,
                'sku_id': sku,
                'quantity_to_move': int(move_qty)
            })
            needed_qty -= move_qty
            if needed_qty <= 0:
                break


In [29]:
redistribution_df = pd.DataFrame(redistribution)

# Final Tables
forecast_df[['store_id', 'sku_id', 'dt', 'forecasted_demand', 'inventory_level', 'alert']].head(10)
redistribution_df.head(10)

# Optional: Save for frontend
# forecast_df.to_csv("forecast_output.csv", index=False)
# redistribution_df.to_csv("redistribution_alerts.csv", index=False)


,from_store,to_store,sku_id,quantity_to_move
0,108,103,PROD003,35
1,100,104,PROD001,58
2,107,104,PROD001,28
3,109,104,PROD001,8
4,100,106,PROD001,3
5,101,102,PROD002,79
6,104,102,PROD002,4
7,100,106,PROD001,29
8,108,103,PROD003,15
9,108,102,PROD003,37


In [43]:
# print("📦 forecast_output.csv")
# print(forecast_df[forecast_df['alert']][['store_id', 'sku_id', 'dt', 'forecasted_demand', 'inventory_level', 'alert']].head())

forecast_output_df = forecast_df[['store_id', 'sku_id', 'dt', 'forecasted_demand', 'inventory_level', 'alert']].head(10)
print(forecast_output_df)


   store_id   sku_id                  dt  forecasted_demand  inventory_level  \
0       106  PROD002 2025-07-13 00:00:00                 36               46   
1       103  PROD003 2025-07-13 01:00:00                 82               47   
2       107  PROD003 2025-07-13 02:00:00                 38               92   
3       104  PROD001 2025-07-13 03:00:00                111               17   
4       106  PROD001 2025-07-13 04:00:00                 77               74   
5       109  PROD003 2025-07-13 05:00:00                 74               95   
6       102  PROD002 2025-07-13 06:00:00                109               26   
7       106  PROD001 2025-07-13 07:00:00                109               80   
8       107  PROD003 2025-07-13 08:00:00                 81               98   
9       104  PROD003 2025-07-13 09:00:00                 42               54   

   alert  
0  False  
1   True  
2  False  
3   True  
4   True  
5  False  
6   True  
7   True  
8  False  
9  False 

In [41]:
# Select relevant columns and create a new DataFrame
forecast_output_df = forecast_df[['store_id', 'sku_id', 'dt', 'forecasted_demand', 'inventory_level', 'alert']].head(10)

# Print it out
print("📦 forecast_output.csv")
print(forecast_output_df)


📦 forecast_output.csv
   store_id   sku_id                  dt  forecasted_demand  inventory_level  \
0       106  PROD002 2025-07-13 00:00:00                 36               46   
1       103  PROD003 2025-07-13 01:00:00                 82               47   
2       107  PROD003 2025-07-13 02:00:00                 38               92   
3       104  PROD001 2025-07-13 03:00:00                111               17   
4       106  PROD001 2025-07-13 04:00:00                 77               74   
5       109  PROD003 2025-07-13 05:00:00                 74               95   
6       102  PROD002 2025-07-13 06:00:00                109               26   
7       106  PROD001 2025-07-13 07:00:00                109               80   
8       107  PROD003 2025-07-13 08:00:00                 81               98   
9       104  PROD003 2025-07-13 09:00:00                 42               54   

   alert  
0  False  
1   True  
2  False  
3   True  
4   True  
5  False  
6   True  
7   True 

In [21]:
# Convert shortage alerts to JSON for frontend use
alerts_json = shortages[['store_id', 'product_id', 'dt', 'inventory', 'forecasted_demand']].to_dict(orient='records')

import json
with open('redistribution_alerts.json', 'w') as f:
    json.dump(alerts_json, f, indent=4)

# Preview
print(json.dumps(alerts_json[:2], indent=2))


TypeError: Object of type Timestamp is not JSON serializable